# Suricata ruleset analysis

This notebook provides some visualizations on a Suricata ruleset.

## Prepare the data

Run suricata to generate a `rules.json` file.

In [ ]:
! #rm data/rules.json
! #suricata --engine-analysis -l data -S paw.rules

Load `rules.json` from ruleset analysis

In [ ]:
import json

rules_json = open('scirius/rules.json', 'r')
ruleset = []
for line in rules_json:
    ruleset.append(json.loads(line))

In [ ]:
#print(json.dumps(ruleset[0], indent=2))


Generate a simple structure with signature and mpm information

In [ ]:
ruleset_mpm =[]
for sig in ruleset:
    if 'mpm' in sig:
        ruleset_mpm.append({'raw': sig['raw'].replace('$','\\$'), 'id': sig['id'], 'msg': sig['msg'], 'proto': sig.get('app_proto', 'raw'), 'buffer': sig['mpm']['buffer'], 'pattern': sig['mpm']['pattern']})

## Find overused patterns

With a lot of signatures attached triggering potential long linear evaluation

In [ ]:
import networkx as nx
import html

G = nx.Graph()

for sig in ruleset_mpm:
    #G.add_node(sig['proto'], type='proto')
    G.add_node(sig['id'], type='sig')
    G.add_node(sig['buffer'], type='buffer')
    pattern = html.escape(sig['pattern'])
    G.add_node(pattern, type='pattern')
    G.add_edge(sig['id'], pattern)
    G.add_edge(pattern, sig['buffer'])
    #G.add_edge(sig['buffer'], sig['proto'])
    #G.add_edge(sig['buffer'], sig['id'])

In [ ]:
import pandas as pd
from ipywidgets import interact, interactive

def get_high_degree(degree):
    high_degree_patterns = []
    pd.set_option("max_colwidth", 440)
    for node in G:
        if G.nodes[node]['type'] == 'pattern':
            if G.degree(node) > degree:
                high_degree_patterns.append([html.unescape(node), G.degree(node) - 1])
    patterns_pd = pd.DataFrame(high_degree_patterns, columns=['pattern', 'degree']).sort_values('degree', ascending=False)
    display(patterns_pd)
    return patterns_pd 

patterns_pd = interactive(get_high_degree, degree=10)
display(patterns_pd)

In [ ]:
from ipywidgets import interact

ruleset_pd = pd.DataFrame(ruleset_mpm)

pd.set_option("max_colwidth", 440)

def display_pattern_sig(pattern='toto'):
    display(ruleset_pd[ruleset_pd.pattern==pattern][["id", "proto", "buffer", "raw"]])

interact(display_pattern_sig, pattern=patterns_pd.result['pattern'].tolist());

## Signatures with application layer and MPM on payload

In [ ]:
app_mpm_payload_pd = ruleset_pd[(ruleset_pd.proto!='raw') & (ruleset_pd.buffer=='payload')]

display(app_mpm_payload_pd['proto'].value_counts())

def display_app_sigs(app='http'):
    display(app_mpm_payload_pd[app_mpm_payload_pd.proto==app])

interact(display_app_sigs, app=app_mpm_payload_pd['proto'].unique());

In [ ]:
import hvplot.networkx as hvnx
colors = []
for node in G:
    if G.nodes[node]['type'] == 'proto':
        colors.append('blue')
    elif G.nodes[node]['type'] == 'sig':
        colors.append('green')
    elif G.nodes[node]['type'] == 'pattern':
        colors.append('black')
    else:
        colors.append('red')
    
pos = nx.spring_layout(G)
hvnx.draw(G, pos, node_color=colors, node_size=40, width=800, height=400)

In [ ]:
#conda install -y hvplot bokeh networkx


In [ ]:
#conda install -y panel=0.12.7
